<a href="https://colab.research.google.com/github/MariaShaiina/hpc-2022/blob/main/Lab3_%20PI%20calc/Lab_3_PI_%D1%81alc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 6132 Шаина Мария

### **Calculation of pi value on GPU with Monte-Carlo method using CUDA**

Загружаем необходимые библиотеки 

In [1]:
import numpy as np
from numba import cuda
from time import time
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import pandas as pd

In [2]:
TPB = 16                     #TPB - treads_per_block
pi_size = 16                 
BPG = int(pi_size/TPB)       #BPG - blocks_per_grid

Функция подсчета значения Пи на CPU



In [3]:
def CPU_calc(pi_size, N):
    res = np.zeros(pi_size)              
    for i in range(pi_size):
        x = np.random.uniform(size=N)
        y = np.random.uniform(size=N)    
        z = x ** 2 + y ** 2 <= 1
        res[i] = 4.0 * sum(z) / N
    return res

Ядро для GPU

In [4]:
@cuda.jit
def pi_calcul(res, rng):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  h = 0
  if i < len(rng):
    x = xoroshiro128p_uniform_float32(rng, i)
    y = xoroshiro128p_uniform_float32(rng, i)
    if x ** 2 + y ** 2 <= 1.0:
      h += 1
  cuda.atomic.add(res, 0, h)

  

Функция, выполняющая вычисление значения Пи на GPU

In [5]:
def GPU_calc(N):

  # create_xoroshiro128p_states - для генерации случайного массива на CUDA
  rng_states = create_xoroshiro128p_states(N, seed=1)
  res = [0]

  dev_res = cuda.to_device(res)
  pi_calcul[N, TPB](dev_res, rng_states)
  tmp_ = dev_res.copy_to_host()
  pi_ = 4 * tmp_[0] / N
  return pi_

In [6]:
# 10 итераций подсчета числа Пи с увеличением значения N на 10000 после каждой итерации
iteration_count = np.arange(1,11,1)
N = 10000

df = {"Время на GPU":np.zeros(len(iteration_count), dtype=float), 
      "Время на CPU":np.zeros(len(iteration_count), dtype=float),
      "Пи на GPU":np.zeros(len(iteration_count), dtype=float),
      "Пи на CPU":np.zeros(len(iteration_count), dtype=float),
      "Ускорение":np.zeros(len(iteration_count), dtype=float),
      "Кол-во точек": np.zeros(len(iteration_count), dtype=int)}

df = pd.DataFrame(df, index=iteration_count)

for i in iteration_count:
  cpu_start = time()
  cpu_pi = CPU_calc(pi_size, N).mean()
  cpu_time = time() - cpu_start

  gpu_start = time()
  gpu_pi = GPU_calc(N)
  gpu_time = time() - gpu_start
    
  df.loc[i, "Время на GPU"] = gpu_time
  df.loc[i, "Время на CPU"] = cpu_time
  df.loc[i, "Пи на GPU"] = gpu_pi
  df.loc[i, "Пи на CPU"] = cpu_pi
  df["Ускорение"] = df["Время на CPU"] / df["Время на GPU"]
  df.loc[i, "Кол-во точек"] = N

  N += 10000

df

,Время на GPU,Время на CPU,Пи на GPU,Пи на CPU,Ускорение,Кол-во точек
1,3.519468,0.526908,3.155200,3.143050,0.149712,10000
2,0.011912,1.360312,3.157800,3.139638,114.198010,20000
3,0.075046,2.279797,3.150800,3.141475,30.378640,30000
4,0.020915,3.798893,3.147100,3.139475,181.638734,40000
5,0.031536,4.962492,3.143040,3.147735,157.360275,50000
6,0.017329,2.670311,3.147067,3.140267,154.090913,60000
7,0.019575,2.129305,3.147543,3.144546,108.776077,70000
8,0.029769,2.325678,3.147450,3.140713,78.124925,80000
9,0.025633,2.610164,3.145378,3.140169,101.829766,90000
10,0.028978,2.961246,3.143720,3.141620,102.189061,100000
